In [ ]:
import os
from google.oauth2 import service_account
from google.cloud import bigquery

# load service account file
# `os.path.expanduser()` converts home dir (~/) to an absolute path
#   change the file path to match your cred file location
cred_file = os.path.expanduser("~/.creds/dsa-deb-sa.json")      
creds = service_account.Credentials.from_service_account_file(cred_file)

# create a connection
# specify the credentials file and the project
project_id = 'deb-01-372116'
client = bigquery.Client(credentials=creds, project=project_id)

print("Successfully created a BiqQuery client")
print(f"Project: {client.project}")

#lists the data sets in project: 'deb-01-372116', 'plants' is infact listed!
print("Listing datasets:")

for dataset in client.list_datasets():
    dataset_id = dataset.dataset_id
    print(f"dataset id: `{dataset_id}`, full_name: `{dataset.full_dataset_id}`, labels (tags): {dataset.labels}")